In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Day Open']    = Calc.groupby(['Symbol','TF','Date'], sort=0)['Open'].transform('first')
    Calc['Day Close']   = Calc.groupby(['Symbol','TF','Date'], sort=0)['Close'].transform('last')

    Calc['ID Chg Bwd']  =                     Calc['Open'] - Calc['Day Open']
    Calc['ID Chg Fwd']  = Calc['Day Close'] - Calc['Open']

    Calc['ID Chg Bwd Abs'] = Calc['ID Chg Bwd'].abs()
    Calc['ID Chg Bwd Pos'] = Calc['ID Chg Bwd'].apply(lambda x: x if (x >= 0) else nan)
    Calc['ID Chg Bwd Neg'] = Calc['ID Chg Bwd'].apply(lambda x: x if (x <  0) else nan)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [43]:
def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 

        Df = Calc[Calc['Time'] <= dt.time(HR,00)]

        pipe.append({
            'Time <': dt.time(HR,00),
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)

STATS(Calc, Col='ID Chg Bwd')

,Time <,count,mean,std,min,25%,50%,75%,max
0,09:00:00,1240,0,0,0,0,0,0,0
1,10:00:00,2480,-2,430,-3113,0,0,20,2818
2,11:00:00,3720,5,663,-3223,-219,0,213,6087
3,12:00:00,4960,-2,822,-5095,-348,0,337,6087
4,13:00:00,6205,-3,936,-5461,-440,0,421,6087
5,14:00:00,7450,-3,1015,-5461,-503,0,488,6087
6,15:00:00,8695,-4,1083,-5461,-568,0,554,6087
7,16:00:00,9940,-5,1146,-6715,-615,0,603,6087
8,17:00:00,11185,-6,1203,-6715,-664,0,650,6288
9,18:00:00,11857,-7,1222,-6715,-680,0,675,6288


In [38]:
STATS(Calc, Col='ID Chg Bwd Abs')

,Time <,count,mean,std,min,25%,50%,75%,max
0,09:00:00,1240,0,0,0,0,0,0,0
1,10:00:00,2480,230,363,0,0,0,363,3113
2,11:00:00,3720,406,524,0,0,215,629,6087
3,12:00:00,4960,533,626,0,0,342,832,6087
4,13:00:00,6205,631,691,0,63,430,968,6087
5,14:00:00,7450,703,732,0,109,495,1069,6087
6,15:00:00,8695,764,768,0,148,562,1153,6087
7,16:00:00,9940,816,804,0,177,608,1216,6715
8,17:00:00,11185,864,837,0,202,660,1283,6715
9,18:00:00,11857,881,847,0,217,676,1303,6715


In [39]:
STATS(Calc, Col='ID Chg Bwd Pos')

,Time <,count,mean,std,min,25%,50%,75%,max
0,09:00:00,1240,0,0,0,0,0,0,0
1,10:00:00,1879,151,308,0,0,0,148,2818
2,11:00:00,2497,307,490,0,0,0,472,6087
3,12:00:00,3108,424,592,0,0,150,686,6087
4,13:00:00,3728,523,665,0,0,265,844,6087
5,14:00:00,4361,598,711,0,0,344,964,6087
6,15:00:00,4997,661,747,0,0,416,1067,6087
7,16:00:00,5633,715,779,0,37,472,1142,6087
8,17:00:00,6273,765,810,0,77,525,1201,6288
9,18:00:00,6628,782,818,0,94,550,1221,6288


In [42]:
STATS(Calc, Col='ID Chg Bwd Neg')

,Time <,count,mean,std,min,25%,50%,75%,max
0,09:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10:00:00,601.0,-478.9,409.4,-3113.0,-650.0,-383.0,-187.0,-5.0
2,11:00:00,1223.0,-610.4,532.6,-3223.0,-855.5,-465.0,-228.0,-5.0
3,12:00:00,1852.0,-716.5,637.8,-5095.0,-991.0,-548.0,-261.0,-5.0
4,13:00:00,2477.0,-794.4,697.9,-5461.0,-1086.0,-620.0,-284.0,-5.0
5,14:00:00,3089.0,-851.7,735.6,-5461.0,-1183.0,-680.0,-305.0,-5.0
6,15:00:00,3698.0,-902.5,774.5,-5461.0,-1238.2,-716.0,-326.0,-5.0
7,16:00:00,4307.0,-948.0,817.1,-6715.0,-1309.0,-749.0,-342.5,-5.0
8,17:00:00,4912.0,-990.5,853.4,-6715.0,-1377.5,-778.5,-354.0,-5.0
9,18:00:00,5229.0,-1006.1,866.7,-6715.0,-1401.0,-792.0,-361.0,-5.0
